In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('tweets_stocks_combined_5mins.csv')

In [14]:
X = df['cleaned_text']
y = df['5mins_price_diff_perc']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)